In [1]:
import torch
import time
import numpy as np
import torch.nn as nn
from torch.nn import functional as F

device = 'cuda' if torch.cuda.is_available() else 'cpu'
##the two hyperparametes
block_size = 8 #length of integers
batch_size = 4 #number of blocks in parallel
max_iters = 1000
learning_rate = 3e-4
eval_iters = 250
dropout = 0.2

In [4]:
with open('wizard_of_oz.txt','r',encoding='utf-8') as f:
    text = f.read()
# print(text[:200])
print(type(text))
chars = sorted(set(text))
vocab_size = len(chars)
print(chars)


<class 'str'>
['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [5]:
string_int = {ch:i for i,ch in enumerate(chars)}
int_string = {i:ch for i,ch in enumerate(chars)}
encode  = lambda s: [string_int[c] for c in s]
decode = lambda l: ''.join([int_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
a = encode('hello')
decode(a)

'hello'

In [6]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]


x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(t,'--when context is', context, 'target =', target)

0 --when context is tensor([1]) target = tensor(1)
1 --when context is tensor([1, 1]) target = tensor(28)
2 --when context is tensor([ 1,  1, 28]) target = tensor(39)
3 --when context is tensor([ 1,  1, 28, 39]) target = tensor(42)
4 --when context is tensor([ 1,  1, 28, 39, 42]) target = tensor(39)
5 --when context is tensor([ 1,  1, 28, 39, 42, 39]) target = tensor(44)
6 --when context is tensor([ 1,  1, 28, 39, 42, 39, 44]) target = tensor(32)
7 --when context is tensor([ 1,  1, 28, 39, 42, 39, 44, 32]) target = tensor(49)


In [7]:
torch_rand1 = torch.rand(10000,10000).to(device)
torch_rand2 = torch.rand(10000,10000).to(device)
np_rand1 = torch.rand(10000,10000)
np_rand2 = torch.rand(10000,10000)
start = time.time()
rand = (torch_rand1 @ torch_rand2)
end = time.time()
time_taken = end-start
print("time taken:", time_taken)

start = time.time()
rand = np.multiply(np_rand1,np_rand2)
end = time.time()
time_taken_np = end-start
print("time taken cpu:",time_taken_np)

time taken: 2.873826026916504
time taken cpu: 0.4612140655517578


In [8]:
%%time
start = time.time()
zeros = torch.zeros(1,1)
end = time.time()
time_taken = end - start
print(time_taken)

0.008967876434326172
CPU times: user 955 µs, sys: 1.7 ms, total: 2.66 ms
Wall time: 10.2 ms


In [11]:
#define a probability tensor
probabilities = torch.tensor([0.1,0.9])
samples  = torch.multinomial(probabilities, num_samples=10, replacement= True)
print(samples)

sample = torch.tensor([10.,10.,10.])
linear = nn.Linear(3,3,bias = False)
print(linear(sample))



tensor([1, 0, 1, 1, 1, 1, 1, 1, 0, 1])
tensor([-7.7470,  0.7305,  7.2389], grad_fn=<SqueezeBackward3>)


In [12]:
# Suppose we have 10 unique words (num_embeddings) and we want to embed them into 5-dimensional vectors (embedding_dim)
embedding = nn.Embedding(num_embeddings=10, embedding_dim=5)

# Create a tensor of indices representing words in a sentence
input_indices = torch.LongTensor([0, 2, 4, 5, 7])

# Retrieve the embeddings for these indices
embedded_words = embedding(input_indices)

print(embedded_words)  # Each row corresponds to the embedding of a word at the respective index

tensor([[-1.4110e+00, -5.3951e-01, -5.2133e-01,  9.6496e-01, -3.1941e-01],
        [-4.8408e-01,  2.9041e+00,  8.7767e-01,  3.2976e-01, -2.7892e-01],
        [ 1.2444e+00,  1.3556e-02, -1.6698e-02, -4.1686e-01,  1.6406e-03],
        [-1.4825e-01,  1.2516e+00,  1.5249e+00,  2.1618e-02, -1.1154e+00],
        [ 4.7466e-01, -1.9139e-01,  1.0134e+00,  4.4341e-02,  1.9125e+00]],
       grad_fn=<EmbeddingBackward0>)


In [13]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
  data = train_data if split == 'train' else val_data
  ix = torch.randint(len(data) - block_size, (batch_size,))
  x = torch.stack([data[i:i+block_size] for i in ix])
  y = torch.stack([data[i+1:i+block_size+1] for i in ix])
  x = x.to(device)
  y = y.to(device)
  return x,y


In [14]:
@torch.no_grad()    ##to disable 
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

In [15]:
class BigramLanguageModel(nn.Module):
  def __init__(self,vocab_size):
    super().__init__()
    self.token_embedding_table = nn.Embedding(vocab_size,vocab_size)
    
  
  def forward(self,index,targets = None):       ##****************
    logits = self.token_embedding_table(index)

    if targets is None:
      loss = None
    else:
      B, T, C = logits.shape
      logits = logits.view(B*T,C)
      targets = targets.view(B*T)
      loss = F.cross_entropy(logits,targets)
    return logits, loss

  def generate(self, index, max_new_tokens):
    for _ in range(max_new_tokens):
      logits,loss = self.forward(index)
      logits = logits[:,-1,:]
      probs = F.softmax(logits,dim = -1)
      index_next = torch.multinomial(probs, num_samples = 1)
      index = torch.cat((index,index_next), dim = 1)
    return index

model = BigramLanguageModel(vocab_size)
m = model.to(device)
context = torch.zeros((1,1), dtype = torch.long, device=device)     ##torch.long - 64bits/8bytes
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


?
6w-UDoRRV1]'0jtOa&w;T)g'c:7zns*V ]OXbw:O)gt-_HAk?,jili X]_M[&6xyrGt4i]"mOg2q,jb,qQQHS -B]5lD!
cEjt1,38w -?v9g,Jgvl4v9&EqcQ4TKZ:1jiNzQvUhl[8hOgRNzZJ[BAI'Ns[t)7&kFpfuXbL.:t_n2z"6iK.u*nhB5AemE cvO?Ey?"9Uhnpjj&[b8cfY-6I'9Az49&[Hacb8VEfY
riW.)k'a,.'t*n9J57'?zZmO"*L*G2TOT5S[4MZ;EO5Ar2pZ()2Y2vzgsRuEd!Jy5SJ6BU)Vd:'_vhnR'RRRplkFRRET3[7Vsgjq*Fm36znzG4H]7P]];dO?vUg,jQ.VU-MK5O."8mMDT:A0:p-N_e&5 zij;BGEQnO)2w;J7iWYO] abIyIq5pDo:1rWEj9?YuOW,qd2xNC]b?Oj
cKwji;T3[I'rjDGH]5Yjp?B(FeLtL xwM1Loy7.lw51yD3xGzDp5(m?
